In [ ]:
# Cell 1 — Setup
from pathlib import Path
import json
import pandas as pd
import numpy as np

# 🔧 Change this to your CSV path
csv_path = Path("videos.csv")  # e.g., "my_youtube.csv"
json_path = Path("youtube-scrape.json")

In [ ]:
# Cell 2 — Load CSV (treat everything as string to avoid ID corruption)
# We'll still parse published_date separately if needed.

# Read all columns as string so we don't lose leading zeros or IDs
df = pd.read_csv(csv_path, dtype=str)

# Normalize column names (strip spaces if any)
df.columns = [c.strip() for c in df.columns]

# Make sure these columns exist (optional; remove/adjust if your CSV differs)
expected_cols = [
    "source","channel_name","channel_id","video_id","title","url","description",
    "published_date","keyword","trust_score","query","view_count","like_count",
    "duration","educational_score"
]
missing = [c for c in expected_cols if c not in df.columns]
if missing:
    print("Warning: Missing columns in CSV:", missing)

df.head()

In [ ]:
# Cell 3 — Clean & convert NaNs to None, and coerce numeric fields if present

def coerce_numeric(val):
    if val is None or (isinstance(val, float) and np.isnan(val)):
        return None
    s = str(val).strip()
    if s == "" or s.lower() == "nan":
        return None
    try:
        # prefer int if it looks like an integer, else float
        if s.isdigit():
            return int(s)
        return float(s)
    except Exception:
        return s  # leave as string if not numeric

def clean_row(row: dict) -> dict:
    out = {}
    for k, v in row.items():
        # Convert NaN/empty to None
        if v is None:
            out[k] = None
            continue
        s = str(v).strip()
        if s == "" or s.lower() == "nan":
            out[k] = None
            continue
        out[k] = s

    # Optional: coerce known numeric fields
    for num_col in ["trust_score", "view_count", "like_count", "duration", "educational_score"]:
        if num_col in out:
            out[num_col] = coerce_numeric(out[num_col])

    return out

records = [clean_row(r) for r in df.to_dict(orient="records")]
len(records)

In [ ]:
# Cell 4 — Append to youtube-scrape.json (create if not exists)
# This writes an array of JSON objects. If the file exists, it loads, appends, de-duplicates by video_id, and saves.

existing = []
if json_path.exists():
    with open(json_path, "r", encoding="utf-8") as f:
        try:
            data = json.load(f)
            if isinstance(data, list):
                existing = data
            else:
                print("Warning: Existing JSON is not a list. Starting fresh array.")
        except json.JSONDecodeError:
            print("Warning: Existing JSON invalid. Starting fresh array.")

# Merge
combined = existing + records

# De-duplicate by `video_id` if present
def dedup_by_key(items, key="video_id"):
    seen = set()
    out = []
    for it in items:
        vid = (it or {}).get(key)
        if vid is not None:
            if vid in seen:
                continue
            seen.add(vid)
        out.append(it)
    return out

combined = dedup_by_key(combined, key="video_id")

# Save
with open(json_path, "w", encoding="utf-8") as f:
    json.dump(combined, f, ensure_ascii=False, indent=2)

print(f"Wrote {len(combined)} total records to {json_path}")

In [ ]:
# Cell 5 — Quick peek
# Show the last few appended entries (optional)
pd.DataFrame(records).tail(5)

In [ ]:
# import youtube-scrape.json file and travers each item in json

import json
import os
import re
import time
import random

# Load the JSON file
with open('youtube-scrape.json', 'r', encoding='utf-8') as f:
    data = json.load(f) # data is a list of dictionaries
print(f"Total videos in JSON: {len(data)}")
# Create a directory to store transcripts if it doesn't exist


In [11]:
from youtube_transcript_api import YouTubeTranscriptApi

# traverse each item in json
for item in data:
    video_id = item.get('video_id')
    if not video_id:
        print("Skipping item with no video_id")
        continue

    if 'content' in item and item['content']:
        print(f"Skipping video_id {video_id} as content already exists")
        continue
    # fetch transcript using youtube-transcript-api
    try:
        transcript = YouTubeTranscriptApi().fetch(video_id)
        t_text = ""
        for entry in transcript:
            t_text += entry.text + " "

        # add t_text to data.content
        item['content'] = t_text.strip()
        # save the updated item back to json file
        with open('youtube-scrape.json', 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False, indent=2)
        time.sleep(10)
        print(f"Fetched transcript for video_id {video_id}")
    except Exception as e:
        print(f"Error fetching transcript for video_id {video_id}: {e}")
        time.sleep(5)
        continue    

Skipping video_id 2njn71TqkjA as content already exists
Skipping video_id ZCKRjP_DMII as content already exists
Skipping video_id JYZpxRy5Mfg as content already exists
Skipping video_id 3hxE7Af98AI as content already exists
Skipping video_id _vDZmVXtA7k as content already exists
Skipping video_id sDo7saKaEys as content already exists
Skipping video_id ImmFkjFm-k0 as content already exists
Skipping video_id M3XZBYVSnJ0 as content already exists
Skipping video_id 128fp0rqfbE as content already exists
Skipping video_id bbxmH_Kj7fk as content already exists
Skipping video_id -EJOO3xAjTk as content already exists
Skipping video_id 2fyX600dL2g as content already exists
Skipping video_id RnvCbquYeIM as content already exists
Fetched transcript for video_id xKxrkht7CpY
Fetched transcript for video_id DW0jTe80kmM
Fetched transcript for video_id WcLlpWmEpQ8
Fetched transcript for video_id dhiWSsKUWEg
Fetched transcript for video_id xy9nj94xvKA
Fetched transcript for video_id fHztd6k5ZXY
Fetched 